In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import numpy as np
import warnings

warnings.filterwarnings('ignore', category = FutureWarning)

# Paths to files

##### Book Crossing

In [ ]:
#Path toward scrapped data (BookCrossing original database completed)
path_scrapper = "/content/drive/MyDrive/ProjetML/Base_BookCrossing/Base_completee/"
file_scrapper = path_scrapper         + "bothWebSites_InternetSearch_AllBooks_BookCrossing.csv"

#Path toward cleaning results
#1st step of cleaning, whithout categories / author gender word 2 vec replacement
file_scrapper_cleaned = path_scrapper + "bothWebSites_InternetSearch_AllBooks_BookCrossing_cleaned.csv"
#Final step of cleaning
file_scrapper_w2v = path_scrapper     + "bothWebSites_InternetSearch_AllBooks_BookCrossing_cleaned_w2v.csv"

In [ ]:
books_scrapped = pd.read_csv(file_scrapper, sep = ';', parse_dates=['Year-Of-Publication'], dtype = {"ISBN_13": str})

len(books_scrapped)

114528

In [ ]:
books_scrapped_cleaned = books_scrapped.copy()

##### Good Books

In [ ]:
#Path toward scrapped data (GoodBooks original database completed)
path_scrapper = "/content/drive/MyDrive/ProjetML/Base_BookCrossing/Base_completee/"
file_scrapper = path_scrapper          + "bothWebSites_InternetSearch_AllBooks_GoodBooks.csv"

#Path toward cleaning results
#1st step of cleaning, whithout categories / author gender word 2 vec replacement
file_scrapper_cleaned = path_scrapper + "bothWebSites_InternetSearch_AllBooks_GoodBooks_cleaned.csv"
#Final step of cleaning
file_scrapper_w2v = path_scrapper     + "bothWebSites_InternetSearch_AllBooks_GoodBooks_cleaned_w2v.csv"

# Transformation of column names to be as our SQL data base
Columns :
    - 'book_id'
    - 'average_rating'
    - 'issue_flag'
    - 'is_active' 
    - 'data_source' 
do not exist in books_scrapped

##### Book crossing

In [ ]:
books_scrapped_cleaned.rename( columns = {'ISBN_10': 'isbn', 'ISBN_13': 'isbn_13', 'Book-Title': 'book_title', \
                                  'Book-Author': 'book_author', 'Year-Of-Publication': 'year_of_publication', \
                                  'Description': 'book_description', 'Language': 'book_language', \
                                  'Pages': 'number_of_pages', 'Awards': 'awards', 'Author\'s genre': 'author_genres', \
                                  'Same serie': 'books_in_series'}, inplace = True )

In [ ]:
books_scrapped_cleaned.to_csv(file_scrapper_cleaned, sep = "|", index = False)

##### Good books

In [ ]:
books_scrapped = pd.read_csv(file_scrapper, sep = '|')
books_scrapped_cleaned = books_scrapped.copy()

In [ ]:
books_scrapped_cleaned.rename( columns = {'ISBN_10': 'isbn', 'ISBN_13': 'isbn_13', 'Book-Title': 'book_title', \
                                  'Book-Author': 'book_author', 'Year-Of-Publication': 'year_of_publication', \
                                  'Description': 'book_description', 'Language': 'book_language', \
                                  'Pages': 'number_of_pages', 'Awards': 'awards', 'Author\'s genre': 'author_genres', \
                                  'Same serie': 'books_in_series'}, inplace = True )

In [ ]:
books_scrapped_cleaned.to_csv(file_scrapper_cleaned, sep = "|", index = False)

# Creation of a second column for the second author gender
*   "author_genres"
*   "author_genres_other"

(several genders can appear in this column, with & as separator)

##### Cleaning of Book Crossing / GoodBooks

In [ ]:
books_scrapped_cleaned = pd.read_csv(file_scrapper_cleaned, sep = '|')

In [ ]:
books_scrapped_cleaned.head(5)

,isbn,isbn_13,OtherID,book_title,book_author,year_of_publication,Publisher,Category,book_description,book_language,Image,number_of_pages,awards,author_genres,books_in_series,average_rating
0,0439023483,9.780439e+12,NaN,The Hunger Games : The First Book of the Hunge...,Suzanne Collins,2008-01-01,NaN,Juvenile Fiction,NaN,en,http://books.google.com/books/content?id=sJdUA...,374,NaN,NaN,NaN,NaN
1,0316015849,9.780316e+12,NaN,Twilight,Stephenie Meyer,2006-01-01,"Little, Brown Books for Young Readers",Young Adult Fiction,With 160 million copies of the Twilight Saga s...,en,NaN,544,"Georgia Peach Book Award (2007), Buxtehuder Bu...",Science Fiction & Fantasy,"['https://www.goodreads.com/book/show/41865', ...",3.61
2,0525478817,9.780525e+12,NaN,The Fault in Our Stars,John Green,2012-01-01,Penguin,Juvenile Fiction,Bestselling author John Green returns with an ...,en,http://books.google.com/books/content?id=Dc2LD...,318,"Georgia Peach Book Award (2013), Buxtehuder Bu...",Literature & Fiction,[],4.19
3,1416524797,9.781417e+12,NaN,Angels & Demons,Dan Brown,2006-01-01,Pocket Books,Fiction,This is the book that started it all: we meet ...,en,NaN,736,Humo's Gouden Bladwijzer (2005),Mystery & Thrillers,"['https://www.goodreads.com/book/show/960', 'h...",3.91
4,0062024035,9.780062e+12,NaN,Divergent,Veronica Roth,2012-01-01,Katherine Tegen Books,Juvenile Fiction,Veronica Roth is the New York Times bestsellin...,en,http://books.google.com/books/content?id=-TUin...,576,"Georgia Peach Book Award (2012), South Carolin...",Science Fiction,['https://www.goodreads.com/book/show/18080920...,4.18


In [ ]:
#new column created
books_scrapped_cleaned.insert(14, "author_genres_other", pd.Series())

#We only deal with not empty cells
aut_gen = books_scrapped_cleaned.author_genres[books_scrapped_cleaned["author_genres"].isna() == False]

for i in aut_gen.index:
    gender = books_scrapped_cleaned["author_genres"][i].strip()

    res_split = gender.strip().split('&')

    #In case a split has been done:
    ##### the second author gender must be copied in author_genres_other column
    ##### and the first author gender only must appear in author_genres column
    if(len(res_split) > 1):
        books_scrapped_cleaned["author_genres_other"][i] = res_split[1].strip()
        books_scrapped_cleaned["author_genres"][i] = res_split[0].strip()       

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value i

In [ ]:
print(len(books_scrapped_cleaned))
print(len(books_scrapped))
books_scrapped_cleaned.head(5)

6414
6414


,isbn,isbn_13,OtherID,book_title,book_author,year_of_publication,Publisher,Category,book_description,book_language,Image,number_of_pages,awards,author_genres,author_genres_other,books_in_series,average_rating
0,0439023483,9.780439e+12,NaN,The Hunger Games : The First Book of the Hunge...,Suzanne Collins,2008-01-01,NaN,Juvenile Fiction,NaN,en,http://books.google.com/books/content?id=sJdUA...,374,NaN,NaN,NaN,NaN,NaN
1,0316015849,9.780316e+12,NaN,Twilight,Stephenie Meyer,2006-01-01,"Little, Brown Books for Young Readers",Young Adult Fiction,With 160 million copies of the Twilight Saga s...,en,NaN,544,"Georgia Peach Book Award (2007), Buxtehuder Bu...",Science Fiction,Fantasy,"['https://www.goodreads.com/book/show/41865', ...",3.61
2,0525478817,9.780525e+12,NaN,The Fault in Our Stars,John Green,2012-01-01,Penguin,Juvenile Fiction,Bestselling author John Green returns with an ...,en,http://books.google.com/books/content?id=Dc2LD...,318,"Georgia Peach Book Award (2013), Buxtehuder Bu...",Literature,Fiction,[],4.19
3,1416524797,9.781417e+12,NaN,Angels & Demons,Dan Brown,2006-01-01,Pocket Books,Fiction,This is the book that started it all: we meet ...,en,NaN,736,Humo's Gouden Bladwijzer (2005),Mystery,Thrillers,"['https://www.goodreads.com/book/show/960', 'h...",3.91
4,0062024035,9.780062e+12,NaN,Divergent,Veronica Roth,2012-01-01,Katherine Tegen Books,Juvenile Fiction,Veronica Roth is the New York Times bestsellin...,en,http://books.google.com/books/content?id=-TUin...,576,"Georgia Peach Book Award (2012), South Carolin...",Science Fiction,NaN,['https://www.goodreads.com/book/show/18080920...,4.18


In [ ]:
books_scrapped_cleaned.to_csv(file_scrapper_cleaned, sep = "|", index = False)

# Cleaning of language column (Book crossing / good books)

##### First some not conform language are manually transformed

In [ ]:
books_scrapped_cleaned = pd.read_csv(file_scrapper_cleaned, sep = '|')

In [ ]:
books_scrapped_cleaned.book_language[books_scrapped_cleaned.book_language == "Catalan; Valencian"] = "ca"

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
books_scrapped_cleaned.book_language[books_scrapped_cleaned.book_language == "English, Middle (1100-1500)"] = "en"

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
books_scrapped_cleaned.book_language[books_scrapped_cleaned.book_language == "Multiple languages"] = "en"

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
books_scrapped_cleaned.book_language[books_scrapped_cleaned.book_language == "zh-CN"] = "zh"

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
books_scrapped_cleaned.book_language[books_scrapped_cleaned.book_language == "zh-TW"] = "zh"

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


##### Then automatic transformation is performed

In [ ]:
!pip install iso-639

     |████████████████████████████████| 174kB 8.4MB/s 
  Created wheel for iso-639: filename=iso_639-0.4.5-cp37-none-any.whl size=169063 sha256=8c534ff588435cdcd87bea4c7f1a2598f9d1d191225d03090583a27264da374f
  Stored in directory: /root/.cache/pip/wheels/52/60/07/73aed7d23ae9b5729970632922ed5e45b535bcd4b8df77ebe9
Successfully built iso-639


In [ ]:
from iso639 import languages

In [ ]:
#When more than two letters to code the language, 
#then perform a transformation into ISO 639 coding 
lines_to_transf = books_scrapped_cleaned.book_language[books_scrapped_cleaned.book_language.str.len() > 2]
lines_to_transf.head()

1287        Arabic
1340        French
1538        French
1981    Portuguese
2007        German
Name: book_language, dtype: object

In [ ]:
for i in lines_to_transf.index:
    books_scrapped_cleaned.book_language[i] = languages.get(name = books_scrapped_cleaned.book_language[i]).alpha2 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
print(len(books_scrapped_cleaned))

6414


In [ ]:
books_scrapped_cleaned.to_csv(file_scrapper_cleaned, sep = "|", index = False)

# Cleaning of awards column

In [ ]:
books_scrapped_cleaned = pd.read_csv(file_scrapper_cleaned, sep = '|')

## Analysis of awards

In [ ]:
#We only deal with not empty cells
awards_lines = books_scrapped_cleaned.author_genres[books_scrapped_cleaned["awards"].isna() == False]

#study of the number of awards inside each cell, saved in "nb_awards"
#study of the kind of awards, saved in "awards_name"
nb_awards = []
awards_name = []

for i in awards_lines.index:

    #first we retrieve the "\n", " " ...
    awards_cleaned = books_scrapped_cleaned["awards"][i].strip()

    #then we split when there are several awards in the same cell
    awards_split = awards_cleaned.split(',')
    l = len(awards_split)

    #27 awards for those books !
    if l == 27:
      print("\n\ntitle {0}, at indice {1}\n".format(books_scrapped_cleaned["book_title"][i], i))
      
    nb_awards.append(l)

    #In case a split has been done:
    if(l > 1):
        for j in range(l):
          #we keep only the name, not the year of obtention
          temp = awards_split[j].strip()
          awards_name.append(temp.split("(")[0].strip())
      



title Harry Potter and the Sorcerer's Stone, at indice 2143



title Harry Potter and the Sorcerer's Stone, at indice 2809



title Harry Potter and the Sorcerer's Stone, at indice 9027



title Harry Potter y la piedra filosofal, at indice 21110



title Harry Potter and the Sorcerer's Stone, at indice 28428



title Harry Potter und der Stein der Weisen, at indice 34539



title Harrius Potter et Philosophi Lapis : (Harry Potter and the Philosopher's Stone), at indice 35740



title Harry Potter and the Philosopher's Stone, at indice 50436



title Harry Potter e la pietra filosofale : romanzo, at indice 51993



title Harry Potter and the Sorcerer's Stone, at indice 52581



title Harry Potter and the Sorcerer's Stone, at indice 55787



title Harry Potter à l'école des sorciers, at indice 64658



title Harry Potter and the Sorcerer's Stone, at indice 72188



title Harry Potter and the Sorcerer's Stone, at indice 77387



title Harry Potter and the Philosopher's Stone : Irish Ga

In [ ]:
pd.Series(nb_awards).value_counts()

1     8858
2     3077
3     1249
4      658
5      641
6      143
7       83
8       69
10      28
12      27
9       19
15      16
21      16
27      16
11      11
16       7
14       5
20       4
19       2
13       2
17       1
dtype: int64

In [ ]:
max(nb_awards)

27

In [ ]:
stats_awards = pd.Series(awards_name).value_counts()
stats_awards

Dorothy Canfield Fisher Children's Book Award Nominee                       428
National Book Award Finalist for Fiction                                    233
National Book Critics Circle Award Nominee for Fiction                      224
Hugo Award Nominee for Best Novel                                           210
Newbery Medal Nominee                                                       202
                                                                           ... 
Wilhelm-Hauff-Preis                                                           1
Lambda Literary Award for Gay Men's Science Fiction/Fantasy                   1
Seiun Award 星雲賞 for Best Translated Short Story for "Story of Your Life"      1
Analog Award Nominee for Best Serial Novel or Novella for "Anasazi"           1
Gordon Montador Award                                                         1
Length: 2033, dtype: int64

In [ ]:
stats_awards[stats_awards.values > 100]

Dorothy Canfield Fisher Children's Book Award Nominee     428
National Book Award Finalist for Fiction                  233
National Book Critics Circle Award Nominee for Fiction    224
Hugo Award Nominee for Best Novel                         210
Newbery Medal Nominee                                     202
Newbery Medal                                             173
Nebula Award Nominee for Best Novel                       173
Los Angeles Times Book Prize Nominee for Fiction          171
Margaret A. Edwards Award                                 166
Hugo Award for Best Novel                                 161
Locus Award Nominee for Best Fantasy Novel                159
World Fantasy Award Nominee for Best Novel                156
Booker Prize Nominee                                      154
Lewis Carroll Shelf Award                                 144
Books I Loved Best Yearly                                 144
Pulitzer Prize for Fiction                                142
Internat

In [ ]:
stats_awards.to_csv(path_scrapper + "awards_names.csv", sep = "|", index = True)

## Cleaning of awards (Book crossing & good books)

In [ ]:
#All awards names previously encountered in scrapped data have been saved in a specific file
stats_awards = pd.read_csv(path_scrapper + "awards_names.csv", sep = "|", index_col = 0)

In [ ]:
stats_awards.head()

,0
Dorothy Canfield Fisher Children's Book Award Nominee,428
National Book Award Finalist for Fiction,233
National Book Critics Circle Award Nominee for Fiction,224
Hugo Award Nominee for Best Novel,210
Newbery Medal Nominee,202


In [ ]:
#The list of awards has previously been saved inside "awards_names.csv", from the most famous to the less
#This function output the most famous awards (according to "awards_names.csv" sort) of the provided argument list

def searchInFamousAwards(list_awards):
  print("\n\n")

  #number of awards in the argument list
  l_entry = len(list_awards)
  #number of existing awards, previously saved inside "awards_names.csv"
  l_ref = len(stats_awards)
  #default value of the awards returned
  awards_returned = ""

  #index of the most famous awards inside "awards_names.csv"
  i = 0

  #among all awards existing, we search for the most famous appearing in list_awards
  #=> reading of existing awards inside "awards_names.csv" is performed, 
  #=> from the most famous (index = 0) to the less (index = l_ref)
  while((awards_returned == "") and (i < l_ref)):
    #if the current existing award is in the provided argument list, then output this award
    if stats_awards.index[i] in list_awards:
      awards_returned = stats_awards.index[i] 
    i = i + 1  

  if(awards_returned == ""):
    #in case where no award has been output, then the first one in the provided argument list is returned
    awards_returned = list_awards[0]

  return  awards_returned


In [ ]:
#new column created in order to keep the most famous award of the provided list
books_scrapped_cleaned["awards_cleaned"] = pd.Series()

#We only deal with not empty cells
awards_lines = books_scrapped_cleaned.author_genres[books_scrapped_cleaned["awards"].isna() == False]

for i in awards_lines.index:
    
    #CLEANING of awards in the current file line 
    #############################################
    #first we remove the "\n", " " ...
    awards_cleaned = []
    awards_cleaned = books_scrapped_cleaned["awards"][i].strip()

    #then we split when there are several awards in the same cell
    awards_split = awards_cleaned.split(',')
    l = len(awards_split)

    #first we clean all the awards (because split introduce " ")
    for j in range(l):
      temp = awards_split[j].strip()
      awards_split[j] = temp.split("(")[0].strip()

    #SELECTION of the most famous one
    #############################################
    if(l > 1):
        #search of the most famous award among the provided argument list (previously cleaned)
        res_awards = searchInFamousAwards(awards_split)
        
        if res_awards != "":
          books_scrapped_cleaned["awards_cleaned"][i] = res_awards
        else:
          books_scrapped_cleaned["awards_cleaned"][i] = awards_split[0]
        

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
print(len(books_scrapped_cleaned))

6414


In [ ]:
books_scrapped_cleaned["awards_cleaned"].value_counts()

Lincoln Award Nominee                                      109
Dorothy Canfield Fisher Children's Book Award Nominee       63
All About Romance                                           51
Rhode Island Teen Book Award Nominee                        42
ALA Alex Award                                              41
                                                          ... 
Harvey Awards for Best New Series                            1
WH Smith Literary Award                                      1
Sakura Medal for Middle School Book                          1
Dayton Literary Peace Prize Nominee for Nonfiction           1
DABWAHA Romance Tournament for Best Romance of the Year      1
Name: awards_cleaned, Length: 509, dtype: int64

In [ ]:
#The previous file column "awards" is then no more useful
del books_scrapped_cleaned["awards"]
books_scrapped_cleaned.rename(columns = {"awards_cleaned": "awards"}, inplace = True)

In [ ]:
books_scrapped_cleaned.head()

,isbn,isbn_13,OtherID,book_title,book_author,year_of_publication,Publisher,Category,book_description,book_language,Image,number_of_pages,author_genres,author_genres_other,books_in_series,average_rating,awards
0,0439023483,9.780439e+12,NaN,The Hunger Games : The First Book of the Hunge...,Suzanne Collins,2008-01-01,NaN,Juvenile Fiction,NaN,en,http://books.google.com/books/content?id=sJdUA...,374,NaN,NaN,NaN,NaN,NaN
1,0316015849,9.780316e+12,NaN,Twilight,Stephenie Meyer,2006-01-01,"Little, Brown Books for Young Readers",Young Adult Fiction,With 160 million copies of the Twilight Saga s...,en,NaN,544,Science Fiction,Fantasy,"['https://www.goodreads.com/book/show/41865', ...",3.61,Books I Loved Best Yearly
2,0525478817,9.780525e+12,NaN,The Fault in Our Stars,John Green,2012-01-01,Penguin,Juvenile Fiction,Bestselling author John Green returns with an ...,en,http://books.google.com/books/content?id=Dc2LD...,318,Literature,Fiction,[],4.19,West Australian Young Readers' Book Award
3,1416524797,9.781417e+12,NaN,Angels & Demons,Dan Brown,2006-01-01,Pocket Books,Fiction,This is the book that started it all: we meet ...,en,NaN,736,Mystery,Thrillers,"['https://www.goodreads.com/book/show/960', 'h...",3.91,NaN
4,0062024035,9.780062e+12,NaN,Divergent,Veronica Roth,2012-01-01,Katherine Tegen Books,Juvenile Fiction,Veronica Roth is the New York Times bestsellin...,en,http://books.google.com/books/content?id=-TUin...,576,Science Fiction,NaN,['https://www.goodreads.com/book/show/18080920...,4.18,Lincoln Award Nominee


In [ ]:
books_scrapped_cleaned.to_csv(file_scrapper_cleaned, sep = "|", index = False)

# Creation of a second column for Category (Book crossing & good books)
*   "Category"
*   "Category_other"

## First we separate categories with "&" in two columns
The column "Category_other" has been created to take the second part of such categories (part after "&")

In [ ]:
books_scrapped_cleaned = pd.read_csv(file_scrapper_cleaned, sep = '|')

In [ ]:
books_scrapped_cleaned.head()

,isbn,isbn_13,OtherID,book_title,book_author,year_of_publication,Publisher,Category,book_description,book_language,Image,number_of_pages,author_genres,author_genres_other,books_in_series,average_rating,awards
0,0439023483,9.780439e+12,NaN,The Hunger Games : The First Book of the Hunge...,Suzanne Collins,2008-01-01,NaN,Juvenile Fiction,NaN,en,http://books.google.com/books/content?id=sJdUA...,374,NaN,NaN,NaN,NaN,NaN
1,0316015849,9.780316e+12,NaN,Twilight,Stephenie Meyer,2006-01-01,"Little, Brown Books for Young Readers",Young Adult Fiction,With 160 million copies of the Twilight Saga s...,en,NaN,544,Science Fiction,Fantasy,"['https://www.goodreads.com/book/show/41865', ...",3.61,Books I Loved Best Yearly
2,0525478817,9.780525e+12,NaN,The Fault in Our Stars,John Green,2012-01-01,Penguin,Juvenile Fiction,Bestselling author John Green returns with an ...,en,http://books.google.com/books/content?id=Dc2LD...,318,Literature,Fiction,[],4.19,West Australian Young Readers' Book Award
3,1416524797,9.781417e+12,NaN,Angels & Demons,Dan Brown,2006-01-01,Pocket Books,Fiction,This is the book that started it all: we meet ...,en,NaN,736,Mystery,Thrillers,"['https://www.goodreads.com/book/show/960', 'h...",3.91,NaN
4,0062024035,9.780062e+12,NaN,Divergent,Veronica Roth,2012-01-01,Katherine Tegen Books,Juvenile Fiction,Veronica Roth is the New York Times bestsellin...,en,http://books.google.com/books/content?id=-TUin...,576,Science Fiction,NaN,['https://www.goodreads.com/book/show/18080920...,4.18,Lincoln Award Nominee


In [ ]:
#new column created
books_scrapped_cleaned.insert(8, "Category_other", pd.Series())

#We only deal with not empty cells
cate_gen = books_scrapped_cleaned.author_genres[books_scrapped_cleaned["Category"].isna() == False]

for i in cate_gen.index:
    cate = books_scrapped_cleaned["Category"][i].strip()
    res_split = cate.split('&')

    #In case a split has been done:
    ##### the second category must be copied in Category_other column
    ##### and the first category only must appear in Category column
    if(len(res_split) > 1):
        books_scrapped_cleaned["Category_other"][i] = res_split[1].strip()
        books_scrapped_cleaned["Category"][i] = res_split[0].strip()       

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarnin

In [ ]:
print(len(books_scrapped_cleaned))

6414


In [ ]:
books_scrapped_cleaned.head()

,isbn,isbn_13,OtherID,book_title,book_author,year_of_publication,Publisher,Category,Category_other,book_description,book_language,Image,number_of_pages,author_genres,author_genres_other,books_in_series,average_rating,awards
0,0439023483,9.780439e+12,NaN,The Hunger Games : The First Book of the Hunge...,Suzanne Collins,2008-01-01,NaN,Juvenile Fiction,NaN,NaN,en,http://books.google.com/books/content?id=sJdUA...,374,NaN,NaN,NaN,NaN,NaN
1,0316015849,9.780316e+12,NaN,Twilight,Stephenie Meyer,2006-01-01,"Little, Brown Books for Young Readers",Young Adult Fiction,NaN,With 160 million copies of the Twilight Saga s...,en,NaN,544,Science Fiction,Fantasy,"['https://www.goodreads.com/book/show/41865', ...",3.61,Books I Loved Best Yearly
2,0525478817,9.780525e+12,NaN,The Fault in Our Stars,John Green,2012-01-01,Penguin,Juvenile Fiction,NaN,Bestselling author John Green returns with an ...,en,http://books.google.com/books/content?id=Dc2LD...,318,Literature,Fiction,[],4.19,West Australian Young Readers' Book Award
3,1416524797,9.781417e+12,NaN,Angels & Demons,Dan Brown,2006-01-01,Pocket Books,Fiction,NaN,This is the book that started it all: we meet ...,en,NaN,736,Mystery,Thrillers,"['https://www.goodreads.com/book/show/960', 'h...",3.91,NaN
4,0062024035,9.780062e+12,NaN,Divergent,Veronica Roth,2012-01-01,Katherine Tegen Books,Juvenile Fiction,NaN,Veronica Roth is the New York Times bestsellin...,en,http://books.google.com/books/content?id=-TUin...,576,Science Fiction,NaN,['https://www.goodreads.com/book/show/18080920...,4.18,Lincoln Award Nominee


In [ ]:
 books_scrapped_cleaned["Category_other"].value_counts()

Autobiography     247
Graphic Novels    165
Economics          80
Relationships      17
Fitness            15
Spirit             13
AUTOBIOGRAPHY       6
Recreation          5
Disciplines         4
RELATIONSHIPS       2
GRAPHIC NOVELS      2
ECONOMICS           2
Home                2
FITNESS             1
Name: Category_other, dtype: int64

In [ ]:
 books_scrapped_cleaned["Category"].head(150)

0         Juvenile Fiction
1      Young Adult Fiction
2         Juvenile Fiction
3                  Fiction
4         Juvenile Fiction
              ...         
145               Adultery
146       Juvenile Fiction
147                Fiction
148              Biography
149                Fiction
Name: Category, Length: 150, dtype: object

## The same for categories with "(...)" (in Category column)
The part between "(...)" has been put in the second column

In [ ]:
#We only deal with not empty cells
cate_gen = books_scrapped_cleaned.Category[books_scrapped_cleaned["Category"].isna() == False]

for i in cate_gen.index:
    cate = books_scrapped_cleaned["Category"][i].strip()
    res_split_ent = cate.split('(')

    if (len(res_split_ent) > 1):
      res_split_out = res_split_ent[1].split(')')

      #In case a split has been done:
      ##### the second term (between "(...)") must be copied in Category_other column
      ##### and the first term remains in Category column (without the second term)
      books_scrapped_cleaned["Category"][i] = res_split_ent[0].strip()
      books_scrapped_cleaned["Category_other"][i] = res_split_out[0].strip()
    

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [ ]:
print(len(books_scrapped_cleaned))

6414


In [ ]:
books_scrapped_cleaned["Category"].head(50)

0        Juvenile Fiction
1     Young Adult Fiction
2        Juvenile Fiction
3                 Fiction
4        Juvenile Fiction
5                 Fiction
6        Large type books
7                 Fiction
8        Juvenile Fiction
9        Juvenile Fiction
10       Juvenile Fiction
11                Fiction
12                Fiction
13                Fiction
14                Fiction
15                Fiction
16                Fiction
17                Fantasy
18              Biography
19                Fantasy
20                Fiction
21       Juvenile Fiction
22            Young Adult
23    Juvenile Nonfiction
24       Juvenile Fiction
25            Young Adult
26    Young Adult Fiction
27                Fiction
28                Fiction
29                Fiction
30                Courage
31                Fiction
32                Fiction
33       Juvenile Fiction
34                 Poetry
35              Biography
36              Biography
37       Juvenile Fiction
38       Juv

## Then removing of "," 
The second term is, as previously, put inside "Category_other" column

In [ ]:
#We only deal with not empty cells
cate_gen = books_scrapped_cleaned.author_genres[books_scrapped_cleaned["Category"].isna() == False]

for i in cate_gen.index:
    cate = books_scrapped_cleaned["Category"][i].strip()
    res_split = cate.split(',')
    second_term = ""

    if (len(res_split) > 1):
      #In case a split has been done:
      ##### and the first term (before ",") remains in Category column (without the second term)
      books_scrapped_cleaned["Category"][i] = res_split[0].strip()

      ##### all the terms appearing after "," must be copied in Category_other column
      for j in range(1,len(res_split)):
        second_term = second_term + res_split[j]
      books_scrapped_cleaned["Category_other"][i] = second_term.strip()

    

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
print(len(books_scrapped_cleaned))

6414


In [ ]:
books_scrapped_cleaned["Category"].value_counts()

Fiction             2090
Juvenile Fiction     956
Fantasy              314
Biography            250
Romance              224
                    ... 
Humorous fiction       1
Actresses              1
Amnesia                1
Roland                 1
Autonomy               1
Name: Category, Length: 391, dtype: int64

In [ ]:
books_scrapped_cleaned["Category_other"].value_counts()

Autobiography     247
Graphic Novels    165
Economics          80
strips etc         22
Relationships      17
                 ... 
France              1
Agent               1
Miles               1
Mass.               1
Dutch               1
Name: Category_other, Length: 61, dtype: int64

In [ ]:
books_scrapped_cleaned.to_csv(file_scrapper_cleaned, sep = "|", index = False)

# Cleaning of isbn numbers

In [ ]:
#We only deal with not empty cells
lines_OK = books_scrapped_cleaned.isbn[books_scrapped_cleaned["isbn"].isna() == False]

for i in lines_OK.index:
    #Unexpected spaces / return are removed
    books_scrapped_cleaned.isbn[i] = books_scrapped_cleaned.isbn[i].split()[0]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
len(books_scrapped_cleaned)

6414

In [ ]:
books_scrapped_cleaned.to_csv(file_scrapper_cleaned, sep = "|", index = False)

# Stemming of columns (Book crossing & good books)
- Category and Category_other
- author_genres and author_genres_other

In [ ]:
books_scrapped_cleaned = pd.read_csv(file_scrapper_cleaned, sep = '|')

In [ ]:
books_scrapped_cleaned.head()

,isbn,isbn_13,OtherID,book_title,book_author,year_of_publication,Publisher,Category,Category_other,book_description,book_language,Image,number_of_pages,author_genres,author_genres_other,books_in_series,average_rating,awards
0,['0439023483'],9.780439e+12,NaN,The Hunger Games : The First Book of the Hunge...,Suzanne Collins,2008-01-01,NaN,juvenile fict,NaN,NaN,en,http://books.google.com/books/content?id=sJdUA...,374,NaN,NaN,NaN,NaN,NaN
1,['0316015849'],9.780316e+12,NaN,Twilight,Stephenie Meyer,2006-01-01,"Little, Brown Books for Young Readers",young adult fict,NaN,With 160 million copies of the Twilight Saga s...,en,NaN,544,science fict,fantasi,"['https://www.goodreads.com/book/show/41865', ...",3.61,Books I Loved Best Yearly
2,['0525478817'],9.780525e+12,NaN,The Fault in Our Stars,John Green,2012-01-01,Penguin,juvenile fict,NaN,Bestselling author John Green returns with an ...,en,http://books.google.com/books/content?id=Dc2LD...,318,literatur,fiction,[],4.19,West Australian Young Readers' Book Award
3,['1416524797'],9.781417e+12,NaN,Angels & Demons,Dan Brown,2006-01-01,Pocket Books,fiction,NaN,This is the book that started it all: we meet ...,en,NaN,736,mysteri,thriller,"['https://www.goodreads.com/book/show/960', 'h...",3.91,NaN
4,['0062024035'],9.780062e+12,NaN,Divergent,Veronica Roth,2012-01-01,Katherine Tegen Books,juvenile fict,NaN,Veronica Roth is the New York Times bestsellin...,en,http://books.google.com/books/content?id=-TUin...,576,science fict,NaN,['https://www.goodreads.com/book/show/18080920...,4.18,Lincoln Award Nominee


##### NLTK stemmer

In [ ]:
import nltk
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [ ]:
#We only deal with not empty cells
lines_OK = books_scrapped_cleaned.Category[books_scrapped_cleaned["Category"].isna() == False]

for i in lines_OK.index:
    w = books_scrapped_cleaned["Category"][i]
    books_scrapped_cleaned["Category"][i] = ps.stem(w)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
books_scrapped_cleaned.Category.head()

0       juvenile fict
1    young adult fict
2       juvenile fict
3             fiction
4       juvenile fict
Name: Category, dtype: object

In [ ]:
#We only deal with not empty cells
lines_OK = books_scrapped_cleaned.Category_other[books_scrapped_cleaned["Category_other"].isna() == False]

for i in lines_OK.index:
    w = books_scrapped_cleaned["Category_other"][i]
    books_scrapped_cleaned["Category_other"][i] = ps.stem(w)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
#We only deal with not empty cells
lines_OK = books_scrapped_cleaned.author_genres[books_scrapped_cleaned["author_genres"].isna() == False]

for i in lines_OK.index:
    w = books_scrapped_cleaned["author_genres"][i]
    books_scrapped_cleaned["author_genres"][i] = ps.stem(w)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
#We only deal with not empty cells
lines_OK = books_scrapped_cleaned.author_genres_other[books_scrapped_cleaned["author_genres_other"].isna() == False]

for i in lines_OK.index:
    w = books_scrapped_cleaned["author_genres_other"][i]
    books_scrapped_cleaned["author_genres_other"][i] = ps.stem(w)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
len(books_scrapped_cleaned)

6414

In [ ]:
books_scrapped_cleaned.to_csv(file_scrapper_cleaned, sep = "|", index = False)

# Removing books duplicates (Book crossing & good books)

##### Search by ISBN

In [ ]:
books_scrapped_cleaned = pd.read_csv(file_scrapper_cleaned, sep = '|')

In [ ]:
len(books_scrapped_cleaned.isbn.unique())

6088

In [ ]:
len(books_scrapped_cleaned)

6414

In [ ]:
#for about 114 000 books, execution in 4 min

#We only deal with not empty cells
lines_OK = books_scrapped_cleaned.isbn[books_scrapped_cleaned.isbn.isna() == False]

#we construct as we go along the list of unique ISBN
listISBN = []

for i in lines_OK.index:
  #in case where this ISBN has already been found (so in listISBN)
  #we have the remove the corresponding line
  if (books_scrapped_cleaned.isbn[i] in listISBN):
    books_scrapped_cleaned.drop(i, axis = 0, inplace = True)
  else:
    listISBN.append(books_scrapped_cleaned.isbn[i])

#when a line is removed, its index also is removed
#so this function renumbers the indexes from 0 to n
books_scrapped_cleaned.reset_index(inplace = True, drop = True)

In [ ]:
len(books_scrapped_cleaned)

6410

In [ ]:
books_scrapped_cleaned.head(1)

,isbn,isbn_13,OtherID,book_title,book_author,year_of_publication,Publisher,Category,Category_other,book_description,book_language,Image,number_of_pages,author_genres,author_genres_other,books_in_series,average_rating,awards
0,0439023483,9.780439e+12,NaN,The Hunger Games : The First Book of the Hunge...,Suzanne Collins,2008-01-01,NaN,juvenile fict,NaN,NaN,en,http://books.google.com/books/content?id=sJdUA...,374,NaN,NaN,NaN,NaN,NaN


In [ ]:
books_scrapped_cleaned.to_csv(file_scrapper_cleaned, sep = "|", index = False)

##### Search by Title + Author

In [ ]:
books_scrapped_cleaned = pd.read_csv(file_scrapper_cleaned, sep = '|')

In [ ]:
print(len(books_scrapped_cleaned))
books_scrapped_cleaned.head(1)

6410


,isbn,isbn_13,OtherID,book_title,book_author,year_of_publication,Publisher,Category,Category_other,book_description,book_language,Image,number_of_pages,author_genres,author_genres_other,books_in_series,average_rating,awards
0,0439023483,9.780439e+12,NaN,The Hunger Games : The First Book of the Hunge...,Suzanne Collins,2008-01-01,NaN,juvenile fict,NaN,NaN,en,http://books.google.com/books/content?id=sJdUA...,374,NaN,NaN,NaN,NaN,NaN


In [ ]:
#for about 114 000 books, execution in 13 min

#We only deal with not empty cells
lines_OK = books_scrapped_cleaned.book_author[books_scrapped_cleaned.book_author.isna() == False]

#save of the duplicates in another dataframe (and later in another csv file)
df_duplicates = pd.DataFrame(columns = books_scrapped_cleaned.columns)

#we construct as we go along the list of unique tuple (Title, Author)
listTitleAuthor = []
nb_duplicates = 0

for i in lines_OK.index:

  #in case where this couple (title, author) has already been found (so in listTitleAuthor)
  #we have the remove the corresponding line
  if ((books_scrapped_cleaned.book_title[i], books_scrapped_cleaned.book_author[i]) in listTitleAuthor):
    #lines removed are saved in another dataframe
    df_duplicates.loc[nb_duplicates] = books_scrapped_cleaned.loc[i]
    nb_duplicates = nb_duplicates + 1

    books_scrapped_cleaned.drop(i, axis = 0, inplace = True)
  else:
    listTitleAuthor.append((books_scrapped_cleaned.book_title[i], books_scrapped_cleaned.book_author[i]))

#when a line is removed, its index also is removed
#so this function renumbers the indexes from 0 to n
books_scrapped_cleaned.reset_index(inplace = True, drop = True)

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            6400, 6401, 6402, 6403, 6404, 6405, 6406, 6407, 6408, 6409],
           dtype='int64', length=6345)


In [ ]:
print(len(df_duplicates))

26


In [ ]:
df_duplicates.book_title.head(3)

0    Harry Potter and the Deathly Hallows
1                              Mockingjay
2                                  Hamlet
Name: book_title, dtype: object

In [ ]:
print(len(books_scrapped_cleaned))
books_scrapped_cleaned.head(1)

6384


,isbn,isbn_13,OtherID,book_title,book_author,year_of_publication,Publisher,Category,Category_other,book_description,book_language,Image,number_of_pages,author_genres,author_genres_other,books_in_series,average_rating,awards
0,0439023483,9.780439e+12,NaN,The Hunger Games : The First Book of the Hunge...,Suzanne Collins,2008-01-01,NaN,juvenile fict,NaN,NaN,en,http://books.google.com/books/content?id=sJdUA...,374,NaN,NaN,NaN,NaN,NaN


In [ ]:
books_scrapped_cleaned.to_csv(file_scrapper_cleaned, sep = "|", index = False)

In [ ]:
#df_duplicates.to_csv(path_scrapper + "Books_Duplicates_BookCrossing.csv", sep = "|", index = False)
df_duplicates.to_csv(path_scrapper + "Books_Duplicates_GoodBooks.csv", sep = "|", index = False)

# Category and author_genres selection

STEP 1 :
All the categories values inside categories columns will be joined : 
=> All unique category values present in the file will be listed

(categories columns are : "Category" and "author_genres", of both books_scrapped_cleaned and df_duplicates)

STEP 2 :
Only a few categories values (among previous list) will be kept

STEP 3 :
The category values not selected will be replaced by one of the previously selected
=> Thanks to a word 2 vec algorithm trained on the file data
=> which will output te closest category value among the previously selected ones

##### STEP 1 : Join

In [ ]:
#STEP 1 on books_scrapped_cleaned & df_duplicates

books_scrapped_cleaned = pd.read_csv(file_scrapper_cleaned, sep = '|')
df_duplicates = pd.read_csv(path_scrapper + "Books_Duplicates.csv", sep = '|')
print(len(books_scrapped_cleaned))
books_scrapped_cleaned.head(1)

99542


,isbn,isbn_13,OtherID,book_title,book_author,year_of_publication,Publisher,Category,Category_other,book_description,book_language,Image,number_of_pages,author_genres,author_genres_other,books_in_series,awards
0,195153448,"9,78E+12",0.0,Classical Mythology,Mark P. O. Morford,2003-01-01,"Oxford University Press, USA",social sci,NaN,Provides an introduction to classical myths pl...,en,http://books.google.com/books/content?id=YABYK...,808.0,NaN,NaN,NaN,NaN


In [ ]:
#INSIDE books_scrapped_cleaned (after duplicates removed)

#indexes are the categories names
stats_cate = books_scrapped_cleaned["Category"].value_counts(sort = True)
cate = stats_cate.index.unique()

stats_aut_genr = books_scrapped_cleaned["author_genres"].value_counts(sort = True)
aut_genr = stats_aut_genr.index.unique()

In [ ]:
#Join of books_scrapped_cleaned["Category"] 
#& 
#books_scrapped_cleaned["author_genres"] unique categories

complete_tab = stats_cate

for g in aut_genr:
    if g in cate:
        complete_tab[g] = stats_aut_genr[g] + stats_cate[g]
    else:
        complete_tab[g] = stats_aut_genr[g]

In [ ]:
complete_tab.shape

(4366,)

In [ ]:
#STEP 1 on df_duplicates

#INSIDE df_duplicates (duplicates saved)

#indexes are the categories names
stats_cate = df_duplicates["Category"].value_counts(sort = True)
cate = stats_cate.index.unique()

stats_aut_genr = df_duplicates["author_genres"].value_counts(sort = True)
aut_genr = stats_aut_genr.index.unique()

In [ ]:
#Join of the previous unique joined books_scrapped_cleaned categories 
#            (books_scrapped_cleaned["Category"] & books_scrapped_cleaned["author_genres"])
#&
#df_duplicates["Category"] saved
#&
#df_duplicates["author_genres"]

for g in aut_genr:
    if g in complete_tab:
        complete_tab[g] = complete_tab[g] + stats_aut_genr[g]
    else:
        complete_tab[g] = stats_aut_genr[g]

for g in cate:
    if g in complete_tab:
        complete_tab[g] = complete_tab[g] + stats_cate[g]
    else:
        complete_tab[g] = stats_cate[g]        

In [ ]:
complete_tab.shape

(4587,)

In [ ]:
complete_tab.sort_values(ascending = False)

fiction                        45933
mysteri                        12284
literatur                      11773
romanc                          9454
science fict                    8089
                               ...  
cambio                             1
social work administration.        1
nemška književnost - romani        1
shop                               1
roma                               1
Name: Category, Length: 4587, dtype: int64

##### STEP 2 : Selection

In [ ]:
#STEP 2 : after previous analysis inside "Books_csv_analysis.ipynb", 
#only the 403 most famous categories are selected

nb_cat_sel = 403
complete_tab.sort_values(ascending = False)[0:nb_cat_sel].to_csv(path_scrapper + "Sel_cat_autgenr.csv", sep = "|")
complete_tab.sort_values(ascending = False)[nb_cat_sel:complete_tab.shape[0]].to_csv(path_scrapper + "Other_cat_autgenr.csv", sep = "|")

##### STEP 3 : creation of train data set for word 2 vec training

In [ ]:
books_scrapped_cleaned = pd.read_csv(file_scrapper_cleaned, sep = '|')
df_duplicates = pd.read_csv(path_scrapper + "Books_Duplicates_GoodBooks.csv", sep = '|')
books_scrapped_cleaned = pd.concat([books_scrapped_cleaned, df_duplicates], ignore_index=True, axis = 0)
len(books_scrapped_cleaned)

6410

In [ ]:
print(len(books_scrapped_cleaned))
books_scrapped_cleaned.head()

6410


,isbn,isbn_13,OtherID,book_title,book_author,year_of_publication,Publisher,Category,Category_other,book_description,book_language,Image,number_of_pages,author_genres,author_genres_other,books_in_series,average_rating,awards
0,0439023483,9.780439e+12,NaN,The Hunger Games : The First Book of the Hunge...,Suzanne Collins,2008-01-01,NaN,juvenile fict,NaN,NaN,en,http://books.google.com/books/content?id=sJdUA...,374,NaN,NaN,NaN,NaN,NaN
1,0316015849,9.780316e+12,NaN,Twilight,Stephenie Meyer,2006-01-01,"Little, Brown Books for Young Readers",young adult fict,NaN,With 160 million copies of the Twilight Saga s...,en,NaN,544,science fict,fantasi,"['https://www.goodreads.com/book/show/41865', ...",3.61,Books I Loved Best Yearly
2,0525478817,9.780525e+12,NaN,The Fault in Our Stars,John Green,2012-01-01,Penguin,juvenile fict,NaN,Bestselling author John Green returns with an ...,en,http://books.google.com/books/content?id=Dc2LD...,318,literatur,fiction,[],4.19,West Australian Young Readers' Book Award
3,1416524797,9.781417e+12,NaN,Angels & Demons,Dan Brown,2006-01-01,Pocket Books,fiction,NaN,This is the book that started it all: we meet ...,en,NaN,736,mysteri,thriller,"['https://www.goodreads.com/book/show/960', 'h...",3.91,NaN
4,0062024035,9.780062e+12,NaN,Divergent,Veronica Roth,2012-01-01,Katherine Tegen Books,juvenile fict,NaN,Veronica Roth is the New York Times bestsellin...,en,http://books.google.com/books/content?id=-TUin...,576,science fict,NaN,['https://www.goodreads.com/book/show/18080920...,4.18,Lincoln Award Nominee


In [ ]:
#We keep only few columns to create the context of training
df1 = books_scrapped_cleaned[["book_description", "Category", "Category_other", "author_genres", "author_genres_other"]]
df1.head()

,book_description,Category,Category_other,author_genres,author_genres_other
0,NaN,juvenile fict,NaN,NaN,NaN
1,With 160 million copies of the Twilight Saga s...,young adult fict,NaN,science fict,fantasi
2,Bestselling author John Green returns with an ...,juvenile fict,NaN,literatur,fiction
3,This is the book that started it all: we meet ...,fiction,NaN,mysteri,thriller
4,Veronica Roth is the New York Times bestsellin...,juvenile fict,NaN,science fict,NaN


In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
stopwords_list = stopwords.words('english')

In [ ]:
stopwords_list[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [ ]:
#with data set of about 100 000 lines, it takes 1 min

#We only deal with not empty cells
lines_OK = df1.book_description[df1.book_description.isna() == False]

for i in lines_OK.index:
    #we get sentence inside book_description column, we remove final '.' and separate all words
    w = df1.book_description[i].strip('.').split()

    #we remove stopwords, words of length < 2 and '...'
    ww = [word.strip(',') for word in w if (word not in stopwords_list) and len(word) > 2 and (word != '...')]

    #We reconstruct a sentence with the remaining words, separated with ','
    df1.book_description[i] = ','.join(ww)


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
df1.book_description.head()

0                                                  NaN
1    With,160,million,copies,Twilight,Saga,sold,wor...
2    Bestselling,author,John,Green,returns,eagerly,...
3    This,book,started,all:,meet,Robert,Langdon,fir...
4    Veronica,Roth,New,York,Times,bestselling,autho...
Name: book_description, dtype: object

In [ ]:
#All those columns informations must be joined inside one column
#str(x) != "nan":     but we join only if column word is not empty
#x.split():           sentences in colum 'book_description' are cut in words
#','.join(x.split()): all those words are joined with (',') to be transformed after in list of list

def WordToJoin(w):
  ww = [x for x in w if str(x) != "nan"]
  return ','.join(ww)

df2 = df1.apply(lambda x: WordToJoin(x), axis = 1)
df2.head()

0                                        juvenile fict
1    With,160,million,copies,Twilight,Saga,sold,wor...
2    Bestselling,author,John,Green,returns,eagerly,...
3    This,book,started,all:,meet,Robert,Langdon,fir...
4    Veronica,Roth,New,York,Times,bestselling,autho...
dtype: object

In [ ]:
#a specific data frame is created with all this train context
df_clean = pd.DataFrame({'clean': df2})
df_clean.head()

,clean
0,juvenile fict
1,"With,160,million,copies,Twilight,Saga,sold,wor..."
2,"Bestselling,author,John,Green,returns,eagerly,..."
3,"This,book,started,all:,meet,Robert,Langdon,fir..."
4,"Veronica,Roth,New,York,Times,bestselling,autho..."


In [ ]:
sent = [row.split(',') for row in df_clean['clean']]
sent[1]

['With',
 '160',
 'million',
 'copies',
 'Twilight',
 'Saga',
 'sold',
 'worldwide',
 'addictive',
 'love',
 'story',
 'teenage',
 'girl',
 'vampire',
 'redefined',
 'romance',
 'generation.',
 'Here&#39;s',
 'book',
 'started',
 'young adult fict',
 'science fict',
 'fantasi']

##### STEP 3 : Skip Gram model training

###### Model creation & training

In [ ]:
!pip install --upgrade gensim

     |████████████████████████████████| 23.9MB 1.3MB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
from gensim.models import Word2Vec

/usr/local/lib/python3.7/dist-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [ ]:
#sg: 0 for CBOW / 1 for skip gram
#window: distance between target word and around words inside context
#
#model = Word2Vec(min_count = 1, vector_size = 50, workers = 3, window = 3, sg = 1)
#


In [ ]:
type(model)

gensim.models.word2vec.Word2Vec

In [ ]:
model.build_vocab(sent)

In [ ]:
model.train(sent, total_examples = model.corpus_count, epochs = model.epochs)

(6891223, 7330380)

In [ ]:
for i in range(2):
  w = books_scrapped_cleaned.Category[i]
  print("Mot original: %s\n" % w)
  print(model.wv[w], "\n\n")
  print(model.wv.most_similar(w, topn=10), "\n\n")

Mot original: social sci

[ 0.9119811  -1.5320835   1.062959    0.87213725 -1.2876016  -0.18897228
  1.1404556   0.13631254 -0.639257    0.27337053 -0.5757122  -0.13806535
  0.47136414  0.09954336 -0.8325257  -0.22439162 -0.3548262  -0.7447521
 -0.68179804 -0.03864497 -0.54879487  0.68530333  1.3502498   0.5358151
 -0.09780243  0.22972333 -0.69660836 -0.3257176  -0.44608447 -0.19397044
  0.17624897  0.01115282  0.30963677 -0.10906821  0.28507406  0.25516146
 -0.71367556 -0.01058627 -0.303773   -0.23146151  0.24702457  0.14805904
 -0.07033689  0.6294008   0.69545573 -0.7897713  -0.48992267  0.2974441
 -0.20150664  0.32073718] 


[('political sci', 0.9123600721359253), ('femin', 0.9025653004646301), ('literary collect', 0.8616941571235657), ('literary critic', 0.8616915345191956), ('psycholog', 0.8612384796142578), ('presid', 0.8598936796188354), ('american literatur', 0.854351818561554), ('refer', 0.8531622290611267), ('architectur', 0.8482690453529358), ('true crim', 0.8466387987136841

In [ ]:
model.save(path_scrapper + "skipGram_cat.model")

###### Model update

In [ ]:
!pip install --upgrade gensim
from gensim.models import Word2Vec

In [ ]:
model = Word2Vec.load(path_scrapper + "skipGram_cat.model")

In [ ]:
#So we need to create a new sent list to update the model
model.build_vocab(sent, update = True)

In [ ]:
model.train(sent, total_examples = len(sent), epochs = model.epochs)

(428225, 467030)

In [ ]:
model.save(path_scrapper + "skipGram_cat.model")

##### STEP 3 : find the category the most close to the selected categories

In [ ]:
#Selected category values
sel_cat = pd.read_csv(path_scrapper + "Sel_cat_autgenr.csv", sep = "|")
sel_cat = sel_cat.rename( columns = {"Unnamed: 0" :"Categories", "Category": "nb_Books_Inside"} )

#Other category values which need to be replaced by one of the selected 
other_cat = pd.read_csv(path_scrapper + "Other_cat_autgenr.csv", sep = "|")
other_cat = other_cat.rename( columns = {"Unnamed: 0" :"Categories", "Category": "nb_Books_Inside"} )

In [ ]:
sel_cat_list = list(sel_cat.Categories)

In [ ]:
print(len(sel_cat_list))
sel_cat_list[0:5]

403


['fiction', 'mysteri', 'literatur', 'romanc', 'science fict']

In [ ]:
other_cat.Categories[1]

'film novel'

In [ ]:
model.wv.most_similar_to_given(key1 = other_cat.Categories[1], keys_list = sel_cat_list)

'fantasy gam'

In [ ]:
#The match between the remaining categories and the closest of the selected ones is saved.
for i in range(other_cat.shape[0]):
  other_cat["Category_Close"][i] = model.wv.most_similar_to_given(key1 = other_cat.Categories[i], keys_list = sel_cat_list)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
other_cat.head()

,Categories,nb_Books_Inside,Category_Close
0,dent,9,baggin
1,film novel,9,fantasy gam
2,caricatures and cartoon,9,sequential art
3,spy thril,9,intelligence servic
4,good and evil,9,dystopia


In [ ]:
other_cat.to_csv(path_scrapper + "Other_cat_autgenr.csv", sep = "|", index = False)

##### STEP 3 : replacment of category values by its closest selected one inside books_scrapped_cleaned

In [ ]:
!pip install --upgrade gensim
from gensim.models import Word2Vec
model = Word2Vec.load(path_scrapper + "skipGram_cat.model")

Requirement already up-to-date: gensim in /usr/local/lib/python3.7/dist-packages (4.0.1)


In [ ]:
print(file_scrapper_cleaned)
books_scrapped_cleaned = pd.read_csv(file_scrapper_cleaned, sep = '|')

sel_cat = pd.read_csv(path_scrapper + "Sel_cat_autgenr.csv", sep = "|")
sel_cat = sel_cat.rename( columns = {"Unnamed: 0" :"Categories", "Category": "nb_Books_Inside"} )
sel_cat_list = list(sel_cat.Categories)

other_cat = pd.read_csv(path_scrapper + "Other_cat_autgenr.csv", sep = "|")
other_cat = other_cat.rename( columns = {"Unnamed: 0" :"Categories", "Category": "nb_Books_Inside"} )

/content/drive/MyDrive/ProjetML/Base_BookCrossing/Base_completee/bothWebSites_InternetSearch_AllBooks_GoodBooks_cleaned.csv


In [ ]:
#We only deal with not empty cells
lines_OK = books_scrapped_cleaned.Category[books_scrapped_cleaned["Category"].isna() == False]

for i in lines_OK.index:
  #if categories in not inside the set selected
  if (books_scrapped_cleaned.Category[i] not in sel_cat_list):

    w = books_scrapped_cleaned.Category[i]

    #Then we search inside 'other_cat' the closest category found thanks to word2vec
    w_close = other_cat[other_cat.Categories == w]["Category_Close"]

    if(len(w_close) != 0):
      #Replacment by one of the selected category values
      books_scrapped_cleaned.Category[i] = w_close.values[0]
    else:
     #in case where this category was not existing for the creation of the model 
     books_scrapped_cleaned.Category[i] = model.wv.most_similar_to_given(key1 = w, keys_list = sel_cat_list)

    

mormon fundament
Series([], Name: Category_Close, dtype: object)
mice
17    dinosaur
Name: Category_Close, dtype: object
general fict
Series([], Name: Category_Close, dtype: object)
model
1143    success
Name: Category_Close, dtype: object
apartment concierg
Series([], Name: Category_Close, dtype: object)
college teach
172    aeronaut
Name: Category_Close, dtype: object
antihero
Series([], Name: Category_Close, dtype: object)
interior decor
783    footbal
Name: Category_Close, dtype: object
betray
595    war novel
Name: Category_Close, dtype: object
breast
733    alcohol
Name: Category_Close, dtype: object
experimental fict
1525    manners and custom
Name: Category_Close, dtype: object
democraci
1078    languag
Name: Category_Close, dtype: object
charm
179    angel
Name: Category_Close, dtype: object
indic fict
1109    hollywood
Name: Category_Close, dtype: object
billionair
2346    horror
Name: Category_Close, dtype: object
fairi
347    diary fict
Name: Category_Close, dtype: object
b

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


african american famili
105    mothers and daught
Name: Category_Close, dtype: object
abbottābād
Series([], Name: Category_Close, dtype: object)
horror fict
3889    english literatur
Name: Category_Close, dtype: object
evolut
148    biolog
Name: Category_Close, dtype: object
animal cal
Series([], Name: Category_Close, dtype: object)
japanese fict
Series([], Name: Category_Close, dtype: object)
christian pilgrims and pilgrimag
507    spain
Name: Category_Close, dtype: object
drunk driv
Series([], Name: Category_Close, dtype: object)
arabic fict
Series([], Name: Category_Close, dtype: object)
explor
90    aeronaut
Name: Category_Close, dtype: object
dark
Series([], Name: Category_Close, dtype: object)
escap
703    compos
Name: Category_Close, dtype: object
teenag
3882    sociolog
Name: Category_Close, dtype: object
children's audiobook
39    accelerated read
Name: Category_Close, dtype: object
belief and doubt
1137    german languag
Name: Category_Close, dtype: object
arabic fict
Series(

In [ ]:
#We only deal with not empty cells
lines_OK = books_scrapped_cleaned.author_genres[books_scrapped_cleaned["author_genres"].isna() == False]

for i in lines_OK.index:
  #if categories in not inside the set selected
  if (books_scrapped_cleaned.author_genres[i] not in sel_cat_list):
    w = books_scrapped_cleaned.author_genres[i]
    
    #Then we search inside 'other_cat' the closest category found thanks to word2vec
    w_close = other_cat[other_cat.Categories == w]["Category_Close"]

    if (len(w_close) != 0):
      #Replacment by one of the selected category values
      books_scrapped_cleaned.author_genres[i] = w_close.values[0]
    else:
     #in case where this category was not existing for the creation of the model 
      books_scrapped_cleaned.author_genres[i] = model.wv.most_similar_to_given(key1 = w, keys_list = sel_cat_list)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
books_scrapped_cleaned.Category.value_counts()

fiction              2135
juvenile fict         967
fantasi               310
biographi             256
romanc                222
                     ... 
disambigu               1
spain                   1
english literatur       1
interpersonal rel       1
aunt                    1
Name: Category, Length: 251, dtype: int64

In [ ]:
books_scrapped_cleaned.to_csv(file_scrapper_w2v, sep = "|", index = False)